In [1]:
import os
import torch
import yaml
import blip
import argparse

from blip.train_generation import BLIPReportGenTrainer
from data.loaders import CxrDataLoader, TransformBuilder

In [2]:
config_path = '../configs/generation.yaml'
config = yaml.load(open(config_path), Loader=yaml.Loader)

In [3]:
args = argparse.Namespace(
    dataset_name='mimic-cxr',
    batch_size=config['batch_size'],
    num_workers=0,
    drop_last=True,
    use_minio=False
)

In [4]:
transform_builder = TransformBuilder(image_size=config['image_size'])

dataloaders = {
    'train': CxrDataLoader(args, split='train', transform=transform_builder.build('train')),
    'val': CxrDataLoader(args, split='val', transform=transform_builder.build('val')),
    'test': CxrDataLoader(args, split='test', transform=transform_builder.build('test')),
}

In [5]:
model_path = os.path.join('./output/generation', 'gen-checkpoint-6000.pt')

In [6]:
model = blip.blip_decoder(
    pretrained=model_path,
    use_custom=True,
    image_size=config['image_size'],
    vit=config['vit'],
    vit_grad_ckpt=config['vit_grad_ckpt'],
    vit_ckpt_layer=config['vit_ckpt_layer'],
    prompt=config['prompt'],
)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Loaded checkpoint from ./output/generation/gen-checkpoint-6000.pt


In [7]:
transform = TransformBuilder(image_size=config['image_size'])
dataloader = CxrDataLoader(args, split='test', transform=transform_builder.build('test'))

batch = next(iter(dataloader))

reports, images = batch['report'], batch['image']
images = images.cpu()
reports = list(reports)

model.eval()
with torch.no_grad():
    preds = model.generate(images)

for pred in preds:
    print(pred)
    print('---')

 no acute cardiopulmonary process. no evidence of pneumonia or vascular congestion. if there is high clinical concern for rib fracture, a dedicated rib series may be obtained. notification : the findings were discussed by dr. _ _ _ with dr. _ _ _ on the telephone on _ _ _ at 11 : 00 am, 5 minutes after discovery of the findings.
---
 ng tube tip is in the stomach. heart size and mediastinum stable. lungs are clear. there is no pleural effusion or pneumothorax. there is no pneumothorax.
---
 low lung volumes with bibasilar atelectasis. no evidence of acute cardiopulmonary process. no pneumonia, vascular congestion, or pleural effusion. if clinical concern for rib fracture, a dedicated rib series may be obtained.
---
 no evidence of acute cardiopulmonary disease. no pneumonia, vascular congestion, or pleural effusion. no evidence of pulmonary edema. no pneumothorax. no evidence of pulmonary edema.
---
 no evidence of acute cardiopulmonary disease. no pneumonia, vascular congestion, or pl

In [ ]:
trainer = BLIPReportGenTrainer(
    model,
    config,
    train_loader=dataloaders['train'],
    val_loader=dataloaders['val'],
    test_loader=dataloaders['test'],
    mixed_precision=False
)

In [ ]:
trainer.train()